### 사전 준비 사항 

#### (1) uv add (터미널)

```bash
uv add rank_bm25
```

#### (2) .env 파일 세팅
```bash
OPENAI_API_KEY = ""
HF_TOKEN = ""
```

#### (3) pdf 파일 세팅
pdf 파일 100개를 `data/raw/files` 에 위치합니다.  
eval 파일(csv 2개, jsonl 1개)을 `data/raw/eval` 에 위치합니다.(*30개 파일 합친 버전)  

#### 실행 방법

1. 처음 1회(커널 새로 시작): 처음부터 끝까지 순서대로 실행
2. exp1 결과 저장 확인
3. <실험 ID 변경> 셀의 exp_id 를 변경 후 새로운 실험 진행 (실험 결과 저장 확인 후 커널 재시작)  
\*커널 재시작하지 않는 경우, 실험 ID 변경 + 실험 진행 섹션 코드만 실행해도 됨.(OOM 발생 가능성이 있어 권장하지 않음.)
4. 원하는 실험로 변경 및 반복

In [1]:
import json, re, unicodedata
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from dotenv import load_dotenv, find_dotenv
from openai import OpenAI
from sentence_transformers import SentenceTransformer

from preprocess.pp_basic import docs, BASE_DIR, EVAL_DIR, GOLD_EVIDENCE_CSV, GOLD_FIELDS_JSONL
from preprocess.rag_experiment import (
    CONFIG, ExperimentSpec, load_questions_df, make_components, RAGExperiment
)

# [추가] RAGAS 통합 실행 함수
from preprocess.ragas_eval import run_experiment_with_ragas

# [추가] 노트북에서 바로 돌릴 수 있게 .env 로드
load_dotenv(find_dotenv(), override=False)

client = OpenAI()

# embed 모델은 커널에서 1번만 로드(중요)
embed_model = SentenceTransformer("nlpai-lab/KoE5")

# gold 로드
gold_evidence_df = pd.read_csv(GOLD_EVIDENCE_CSV)

def load_gold_fields_jsonl(path):
    rows = []
    with open(path, "r", encoding="utf-8", errors="replace") as f:
        for line in f:
            if line.strip():
                rows.append(json.loads(line))
    out = []
    for r in rows:
        iid = r["instance_id"]
        doc_id = r.get("doc_id", "")
        fields = r.get("fields", {}) or {}
        for k, v in fields.items():
            out.append({"instance_id": iid, "doc_id": doc_id, "field": k, "gold": v})
    return pd.DataFrame(out)

gold_fields_df = load_gold_fields_jsonl(GOLD_FIELDS_JSONL)

questions_df = load_questions_df()

print("gold_evidence_df:", gold_evidence_df.shape)
print("gold_fields_df:", gold_fields_df.shape)
print("questions_df:", questions_df.shape)
print("n_docs:", len(docs))

/home/ohs3201/codeit/codeit-part3-team4/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


gold_evidence_df: (630, 5)
gold_fields_df: (630, 4)
questions_df: (311, 5)
n_docs: 100


In [2]:
# 평가 문서 필터(커널당 1회)
def name_key(s: str) -> str:
    s = unicodedata.normalize("NFC", str(s)).strip()
    s = unicodedata.normalize("NFKC", s)
    s = re.sub(r"\s+", " ", s)
    return s

gold_doc_key_set = set(name_key(x) for x in gold_fields_df["doc_id"].astype(str).unique())
EVAL_DOCS = [p for p in docs if name_key(p.name) in gold_doc_key_set]

print("Eval docs:", len(EVAL_DOCS))

Eval docs: 30


### 실험 ID 변경

In [3]:
RUN_EXP_ID = 3   # 1~18
N_DOCS = 5       # 디버깅은 1~5 추천, 전체는 None 또는 아래 라인 변경

# N개 문서 또는 전체 문서 테스트
# RUN_DOCS = EVAL_DOCS[:N_DOCS]
RUN_DOCS = EVAL_DOCS

# exp table (18개)
SPECS = {
    1:  ("C1","R1","G1"),  2:  ("C1","R1","G2"),
    3:  ("C1","R2","G1"),  4:  ("C1","R2","G2"),
    5:  ("C1","R3","G1"),  6:  ("C1","R3","G2"),
    7:  ("C2","R1","G1"),  8:  ("C2","R1","G2"),
    9:  ("C2","R2","G1"),  10: ("C2","R2","G2"),
    11: ("C2","R3","G1"),  12: ("C2","R3","G2"),
    13: ("C3","R1","G1"),  14: ("C3","R1","G2"),
    15: ("C3","R2","G1"),  16: ("C3","R2","G2"),
    17: ("C3","R3","G1"),  18: ("C3","R3","G2"),
    19: ("C4","R1","G1"),  20: ("C4","R1","G2"),
    21: ("C4","R2","G1"),  22: ("C4","R2","G2"),
    23: ("C4","R3","G1"),  24: ("C4","R3","G2"),
}

c, r, g = SPECS[RUN_EXP_ID]
spec = ExperimentSpec(exp_id=RUN_EXP_ID, chunker=c, retriever=r, generator=g)
print("Running spec:", spec)
print("RUN_DOCS:", len(RUN_DOCS))

# (선택) 실험 컨텍스트 cap 조정하고 싶은 경우 주석 해제 및 수정
# CONFIG["max_context_chars"] = 4000

# [추가] RAGAS 실행 여부 / 지표 / 평가자 모델 (원하는 대로 수정)
RUN_RAGAS = True
RAGAS_EVALUATOR_MODEL = "gpt-5-mini"
RAGAS_METRICS = ["faithfulness", "context_precision", "answer_correctness"]

Running spec: ExperimentSpec(exp_id=3, chunker='C1', retriever='R2', generator='G1')
RUN_DOCS: 30


### 디버깅 사전 설정(선택)

In [4]:
# (옵션) sentinel 모니터링용
SENT_NOT_FOUND = "NOT_FOUND"
SENT_GEN_FAIL = "GEN_FAIL"

def count_sentinels(pred_map: dict) -> dict:
    if not isinstance(pred_map, dict):
        return {"n_keys": 0, "n_not_found": 0, "n_gen_fail": 0}

    vals = [str(v).strip() for v in pred_map.values()]
    vals_l = [v.lower() for v in vals]

    n_nf = sum(v in {"not_found", "notfound"} for v in vals_l)
    n_gf = sum(v == "gen_fail" for v in vals_l)
    return {"n_keys": len(vals), "n_not_found": n_nf, "n_gen_fail": n_gf}

### 실험 수행 및 결과 저장

In [5]:
chunker, retriever, generator = make_components(spec, embed_model=embed_model, client=client)
rag = RAGExperiment(chunker=chunker, retriever=retriever, generator=generator, questions_df=questions_df)

rows = []
for doc_path in tqdm(RUN_DOCS, desc=f"Exp {spec.exp_id} docs"):
    m = rag.run_single_doc_metrics(
        doc_path,
        gold_fields_df=gold_fields_df,
        gold_evidence_df=gold_evidence_df,
        top_k=CONFIG["top_k"],
        sim_threshold=80,
    )
    m["exp_id"] = spec.exp_id
    m["chunker"] = spec.chunker
    m["retriever"] = spec.retriever
    m["generator"] = spec.generator
    rows.append(m)

doc_df = pd.DataFrame(rows)

# exp-level average
avg = doc_df[["ret_recall","ret_mrr","gen_fill","gen_match","gen_sim"]].mean(numeric_only=True)
exp_df = pd.DataFrame([{
    "exp_id": spec.exp_id,
    "chunk": spec.chunker,
    "retriever": spec.retriever,
    "model": spec.generator,
    "n_docs": len(doc_df),
    **{k: float(avg[k]) for k in avg.index},
}])

display(exp_df.round(4))

out_dir = BASE_DIR / "outputs"
out_dir.mkdir(parents=True, exist_ok=True)

exp_out = out_dir / f"exp{spec.exp_id:02d}_explevel.csv"
exp_df.to_csv(exp_out, index=False, encoding="utf-8-sig")
print("Saved:", exp_out)

Exp 3 docs: 100%|██████████| 30/30 [09:05<00:00, 18.19s/it]


,exp_id,chunk,retriever,model,n_docs,ret_recall,ret_mrr,gen_fill,gen_match,gen_sim
0,3,C1,R2,G1,30,0.854,0.4926,1.0,0.2296,43.1427


Saved: /home/ohs3201/codeit/codeit-part3-team4/outputs/exp03_explevel.csv


In [6]:
# 문서별 pred_map 저장 (옵션)
pred_dir = out_dir / f"exp{spec.exp_id:02d}_pred_maps"
pred_dir.mkdir(parents=True, exist_ok=True)

saved = 0
for _, row in doc_df.iterrows():
    doc_id = str(row["doc_id"])
    pred_map = row.get("pred_map", None)
    if isinstance(pred_map, dict):
        # 파일명 안전화
        safe = re.sub(r"[\\/:*?\"<>|]", "_", doc_id)
        path = pred_dir / f"{safe}.json"
        with open(path, "w", encoding="utf-8") as f:
            json.dump(pred_map, f, ensure_ascii=False, indent=2)
        saved += 1

print("Saved pred_maps:", saved, "->", pred_dir)

Saved pred_maps: 30 -> /home/ohs3201/codeit/codeit-part3-team4/outputs/exp03_pred_maps


### 디버깅(선택)

In [7]:
DEBUG = True  # 필요할 때만 True

if DEBUG:
    print("last_debug:", getattr(rag.generator, "last_debug", None))

    raw = getattr(rag.generator, "last_raw_text", "") or ""
    print("raw_text_len:", len(raw.strip()))
    print("raw_text_preview:\n", raw[:600])

    d = getattr(rag.generator, "last_resp_dump", None)
    print("dump is None?", d is None)
    if isinstance(d, dict):
        # responses API는 output_text가 별도 필드로 있을 수 있음(덤프엔 없을 때도 있음)
        print("top keys:", list(d.keys())[:30])
        print("status:", d.get("status"))
        usage = d.get("usage") or {}
        print("usage.output_tokens:", usage.get("output_tokens"))
        out = d.get("output")
        if isinstance(out, list):
            print("output item types:", [x.get("type") for x in out if isinstance(x, dict)])

last_debug: {'model': 'gpt-5-mini', 'n_questions': 21, 'context_len': 6000, 'max_context_chars': 6000, 'prompt_len': 7856, 'response_status': 'completed', 'output_tokens': 683, 'output_text_repr': '\'{"project_name":"국립민속박물관 민속아카이브 자료관리시스템 기능개선","agency":"국립민속박물관 유물과학과","purpose":"민속아카이브 자료관리시스템 업무 프로세스 강화, 상태 모니터링 기능 강화, 자료 관리 기능 강화 및 사용자 편의성 개선","budget":"금77,000,000원(금칠천칠백만원, 부가세 포함)","contract\'', 'exception': None, 'parse_error': None}
raw_text_len: 1359
raw_text_preview:
 {"project_name":"국립민속박물관 민속아카이브 자료관리시스템 기능개선","agency":"국립민속박물관 유물과학과","purpose":"민속아카이브 자료관리시스템 업무 프로세스 강화, 상태 모니터링 기능 강화, 자료 관리 기능 강화 및 사용자 편의성 개선","budget":"금77,000,000원(금칠천칠백만원, 부가세 포함)","contract_type":"제한경쟁입찰(협상에 의한 계약)","deadline":"NOT_FOUND","duration":"계약 후 2024.12.20.(금)까지","requirements_must":"행정기관 및 공공기관 정보시스템 구축·운영 지침(행정안전부 고시 제2023-27호) 준수, 개발프레임워크(eGovFrame) 우선 고려, 원격지 개발 보안관리대책, 개인정보보호 및 기밀보안 체계 제시","eval_items":"기술평가(90%) / 가격평가(10%)","price_eval":"가격평가(10%) (평가방식: 가격평가로 포함)","eligibility":"조달청 

In [8]:
# (옵션) GEN_FAIL/NOT_FOUND 비율 빠르게 보기: 마지막 doc 1개 기준
try:
    last_row = doc_df.iloc[-1].to_dict()
    pm = last_row.get("pred_map")
    print("pred_map sentinel counts:", count_sentinels(pm))
except Exception as e:
    print("pred_map sentinel counts: skipped:", repr(e))

pred_map sentinel counts: {'n_keys': 21, 'n_not_found': 1, 'n_gen_fail': 0}


### RAGAS도 “같은 spec/RUN_DOCS/CONFIG”로 한 번에 계산

In [9]:
if RUN_RAGAS:
    # 핵심: 같은 spec, 같은 RUN_DOCS, 같은 embed_model, 같은 client, 같은 CONFIG로 실행
    res = run_experiment_with_ragas(
        spec=spec,
        run_docs=list(RUN_DOCS),
        gold_fields_jsonl_path=GOLD_FIELDS_JSONL,
        embed_model=embed_model,
        client=client,
        evaluator_model=RAGAS_EVALUATOR_MODEL,
        ragas_metrics=RAGAS_METRICS,
        compute_baseline_doc_metrics=False,   # 이미 doc_df를 위에서 만들었으니 중복 계산 방지
        gold_evidence_df=None,                # compute_baseline_doc_metrics=False면 필요 없음
    )

    # 질문 단위 RAGAS
    ragas_sample_df = res.ragas_sample_df
    display(ragas_sample_df.head())

    # 문서 단위 평균 RAGAS
    ragas_doc_df = res.ragas_doc_df
    display(ragas_doc_df.sort_values("faithfulness").head(10))

    # exp 단위 평균 RAGAS
    ragas_exp_df = res.ragas_exp_df
    display(ragas_exp_df)

    # (선택) exp_df(기존)와 ragas_exp_df(새) 한 줄로 합쳐보기
    merged_exp = exp_df.merge(ragas_exp_df, on="exp_id", how="left")
    display(merged_exp.round(4))

    # (선택) 저장
    ragas_out_csv = out_dir / f"exp{spec.exp_id:02d}_ragas_explevel.csv"
    merged_exp.to_csv(ragas_out_csv, index=False, encoding="utf-8-sig")
    print("Saved:", ragas_out_csv)
else:
    print("RUN_RAGAS=False (skip)")

RAG + RAGAS | exp 3: 100%|██████████| 30/30 [09:10<00:00, 18.37s/it]
GPT-5 judge scoring: 100%|██████████| 604/604 [13:38<00:00,  1.36s/it]


,exp_id,chunker,retriever,generator,doc_id,field,user_input,faithfulness,context_precision,answer_correctness
0,3,C1,R2,G1,(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf,project_name,사업(용역)명은 무엇인가?,0.90,0.8,1.00
1,3,C1,R2,G1,(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf,agency,발주 기관(수요기관)은 어디인가?,0.10,0.2,0.00
2,3,C1,R2,G1,(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf,purpose,사업 목적(추진 배경)은 무엇인가?,0.95,0.9,0.95
3,3,C1,R2,G1,(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf,budget,총 사업 예산(사업비)은 얼마인가?,0.00,0.1,0.00
4,3,C1,R2,G1,(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf,contract_type,계약 방식(일반경쟁/제한경쟁/협상에 의한 계약 등)은 무엇인가?,0.00,0.2,0.00


,doc_id,faithfulness,context_precision,answer_correctness
0,(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf,0.302381,0.366667,0.330556
8,경상북도 봉화군_봉화군 재난통합관리시스템 고도화 사업(협상)(긴급).pdf,0.311905,0.516667,0.285000
4,2025 구미 아시아육상경기선수권대회 조직위원회_2025 구미아시아육상경.pdf,0.333333,0.357143,0.322222
2,(사）한국대학스포츠협의회_KUSF 체육특기자 경기기록 관리시스템 개발.pdf,0.478571,0.554762,0.390000
22,나노종합기술원_스마트 팹 서비스 활용체계 구축관련 설비온라인 시스.pdf,0.504286,0.514286,0.541667
21,그랜드코리아레저(주)_2024년도 GKL 그룹웨어 시스템 구축 용역.pdf,0.526190,0.616667,0.508333
24,대검찰청_아태 사이버범죄 역량강화 허브(APC-HUB) 홈페이지 및 온라인 교.pdf,0.550000,0.452381,0.628947
25,대전대학교_대전대학교 2024학년도 다층적 융합 학습경험 플랫폼(MILE) 전.pdf,0.569048,0.590476,0.431579
15,국립인천해양박물관_국립인천해양박물관 해양자료관리시스템 구축 용.pdf,0.573810,0.561905,0.500000
6,경기도 평택시_2024년도 평택시 버스정보시스템(BIS) 구축사업.pdf,0.576190,0.628571,0.438889


,exp_id,faithfulness,context_precision,answer_correctness
0,3,0.615944,0.652185,0.571038


,exp_id,chunk,retriever,model,n_docs,ret_recall,ret_mrr,gen_fill,gen_match,gen_sim,faithfulness,context_precision,answer_correctness
0,3,C1,R2,G1,30,0.854,0.4926,1.0,0.2296,43.1427,0.6159,0.6522,0.571


Saved: /home/ohs3201/codeit/codeit-part3-team4/outputs/exp03_ragas_explevel.csv


In [10]:
doc_compare = doc_df.merge(ragas_doc_df, on="doc_id", how="left")
display(doc_compare[["doc_id","gen_match","gen_sim","faithfulness","context_precision","answer_correctness"]].head())

,doc_id,gen_match,gen_sim,faithfulness,context_precision,answer_correctness
0,(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf,0.095238,23.678638,0.302381,0.366667,0.330556
1,(사)부산국제영화제_2024년 BIFF & ACFM 온라인서비스 재개발 및 행사지원...,0.200000,41.935614,0.714286,0.780952,0.623421
2,(사）한국대학스포츠협의회_KUSF 체육특기자 경기기록 관리시스템 개발.pdf,0.050000,18.617800,0.478571,0.554762,0.390000
3,(재)예술경영지원센터_통합 정보시스템 구축 사전 컨설팅.pdf,0.150000,38.978738,0.630952,0.695238,0.569444
4,2025 구미 아시아육상경기선수권대회 조직위원회_2025 구미아시아육상경.pdf,0.200000,30.709622,0.333333,0.357143,0.322222
